In [1]:
import requests

url = 'https://github.com/opencv/opencv_zoo/blob/main/models/face_recognition_sface/face_recognition_sface_2021dec.onnx?raw=true'
local_path = 'face_recognition_sface_2021dec.onnx'

response = requests.get(url)
with open(local_path, 'wb') as f:
    f.write(response.content)

print(f'Model downloaded and saved as {local_path}')


Model downloaded and saved as face_recognition_sface_2021dec.onnx


In [2]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

def visualize(input, faces, thickness=2):
    if faces[1] is not None:
        coords = faces[1][0].astype(np.int32)
        cv.rectangle(input, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), (0, 255, 0), thickness)
        cv.circle(input, (coords[4], coords[5]), 2, (255, 0, 0), thickness)
        cv.circle(input, (coords[6], coords[7]), 2, (0, 0, 255), thickness)
        cv.circle(input, (coords[8], coords[9]), 2, (0, 255, 0), thickness)
        cv.circle(input, (coords[10], coords[11]), 2, (255, 0, 255), thickness)
        cv.circle(input, (coords[12], coords[13]), 2, (0, 255, 255), thickness)

detector = cv.FaceDetectorYN.create("face_detection_yunet_2023mar.onnx","",(320, 320),0.8,0.3,5000)
recognizer = cv.FaceRecognizerSF.create(local_path,"")

input_image = cv.imread('matin.jpg')
input_image_copy = input_image.copy()
height, width, _ = input_image.shape
detector.setInputSize((width, height))
face1 = detector.detect(input_image)
visualize(input_image_copy, face1)
assert face1[1] is not None, 'cannot find even a image'
face1_crop = recognizer.alignCrop(input_image, face1[1][0])
face1_feature = recognizer.feature(face1_crop)

webcam = cv.VideoCapture('matin_face3.MP4')
while True:
    ok, frame = webcam.read()
    if not ok:
        break
    frame = cv.flip(frame, 1)
    frame_copy = frame.copy()
    frame_width, frame_height = frame.shape[1],frame.shape[0]
    detector.setInputSize((frame_width, frame_height))
    face_frame = detector.detect(frame)
    color = (255,0,0)
    message = 'cannot find even an face'
    if face_frame[1] is not None:
        for face in face_frame[1]:
            visualize(frame_copy,  (None, [face]))
            face_frame_crop = recognizer.alignCrop(frame, face)
            face_frame_feature = recognizer.feature(face_frame_crop)
            cosine_similarity_threshold = 0.363
            cosine_score = recognizer.match(face1_feature, face_frame_feature, cv.FaceRecognizerSF_FR_COSINE)
            message = 'you are not matin'
            color = (0, 0, 255)
            if cosine_score >= cosine_similarity_threshold:
                message = 'you are matin'
                color = (0, 255, 0)
            coords = face[:-1].astype(np.int32)
            p1 = coords[0], coords[1]
            p2 = (coords[0] + coords[2], coords[1] + coords[3])
            cv.rectangle(frame, p1, p2, color, 3)
            cv.putText(frame, message, (coords[0], coords[1] - 5), cv.FONT_HERSHEY_COMPLEX, 0.5, color, 2)
            cv.putText(frame, str(cosine_score), (coords[0], coords[1] - 20), cv.FONT_HERSHEY_COMPLEX, 0.5, color, 2)
    else:
        cv.putText(frame,message,(30,30),cv.FONT_HERSHEY_COMPLEX,1,color,2)
    cv.imshow('frame', frame)
    if cv.waitKey(1) & 0xff == 27:
        break

cv.destroyAllWindows()
webcam.release()
